In [2]:
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED
import numpy as np 
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re

In [3]:
final_data = []
year = 2020
weeks = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

for week in weeks:
    url = f'https://www.espn.com/nfl/scoreboard/_/week/{week}/year/{year}/seasontype/2'
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    week_data = []
    for board in soup.select('.ScoreboardScoreCell'):
        title = board.find_previous(class_='Card__Header__Title').text
        teams = [t.text for t in board.select('.ScoreCell__TeamName')]
        scores = [s.text for s in board.select('.ScoreCell__Score')] or ['-', '-']

        week_data.append((week, title, teams[0], scores[0], teams[1], scores[1]))

    final_data.extend(week_data)

df_scores_2020 = pd.DataFrame(final_data, columns=['Week', 'Date', 'Team 1', 'Score 1', 'Team 2', 'Score 2'])
df_scores_2020 = df_scores_2020.rename(columns={'Team 1': 'Away Team', 'Score 1': 'Away Score', 'Team 2': 'Home Team', 'Score 2':'Home Score'})
df_scores_2020['Final Score'] = df_scores_2020['Away Score'] + ' - ' + df_scores_2020['Home Score']
#print(df.to_markdown(index=False))
# Replace missing scores with NaN and dtype convert object to int 
df_scores_2020['Away Score'] = df_scores_2020['Away Score'].replace('-', np.nan)
df_scores_2020['Home Score'] = df_scores_2020['Home Score'].replace('-', np.nan)
df_scores_2020['Away Score'] = pd.to_numeric(df_scores_2020['Away Score'])
df_scores_2020['Home Score'] =  pd.to_numeric(df_scores_2020['Home Score'])

# Calculate total points
df_scores_2020['Total Points'] = df_scores_2020['Away Score'] + df_scores_2020['Home Score']

# Calculate winner
df_scores_2020['Winner'] = df_scores_2020.apply(lambda row: row['Away Team'] if row['Away Score'] > row['Home Score'] else row['Home Team'], axis=1)

# Calculate score differential
df_scores_2020['Away Score Differential'] = df_scores_2020['Home Score'] - df_scores_2020['Away Score']
df_scores_2020['Home Score Differential'] = df_scores_2020['Away Score'] - df_scores_2020['Home Score']
df_scores_2020['Matchup'] = df_scores_2020['Away Team'] + ' @ ' + df_scores_2020['Home Team']
df_scores_2020['Date'] = pd.to_datetime(df_scores_2020['Date'], format='%A, %B %d, %Y').dt.strftime('%Y-%m-%d')
# Use a lambda function to create the 'Home_Away_Winner' column
df_scores_2020['Home_Away_Winner'] = df_scores_2020.apply(lambda row: 'Home' 
                                                          if row['Winner'] == row['Home Team'] 
                                                          else 'Away' if row['Winner'] == row['Away Team'] else 'No Winner', axis=1)
df_scores_2020.tail()

,Week,Date,Away Team,Away Score,Home Team,Home Score,Final Score,Total Points,Winner,Away Score Differential,Home Score Differential,Matchup,Home_Away_Winner
251,17,2021-01-03,Cardinals,7,Rams,18,7 - 18,25,Rams,11,-11,Cardinals @ Rams,Home
252,17,2021-01-03,Seahawks,26,49ers,23,26 - 23,49,Seahawks,-3,3,Seahawks @ 49ers,Away
253,17,2021-01-03,Saints,33,Panthers,7,33 - 7,40,Saints,-26,26,Saints @ Panthers,Away
254,17,2021-01-03,Titans,41,Texans,38,41 - 38,79,Titans,-3,3,Titans @ Texans,Away
255,17,2021-01-03,Washington,20,Eagles,14,20 - 14,34,Washington,-6,6,Washington @ Eagles,Away


In [4]:
#import consensus data for spreads 
game_dates = list(df_scores_2020['Date'].unique())
uri_list = []
uri = "https://contests.covers.com/consensus/topconsensus/nfl/overall/"

for date in game_dates:
    uri_list.append(uri + date)

spreads = []
for item in uri_list:
    try:
        res = requests.get(item)
        soup = BeautifulSoup(res.content, 'lxml')
        table = soup.find_all('table')[0]
        temp_df = pd.read_html(str(table))[0]  # Obtain the first table from the list
        temp_df['Date'] = re.search(r'\d{4}-\d{2}-\d{2}', item).group()  # Add the corresponding date column

        spreads.append(temp_df)
    except IndexError:
        print(f"IndexError occurred for URL: {item}")
        continue
    except Exception as e:
        print(f"An error occurred for URL: {item}")
        print(f"Error details: {str(e)}")
        continue

df_spreads_2020 = pd.concat(spreads, ignore_index=True)

IndexError occurred for URL: https://contests.covers.com/consensus/topconsensus/nfl/overall/2020-12-02


In [5]:
#clean matchup column 
df_spreads_2020['Matchup'] = df_spreads_2020['Matchup'].str[4:].str.upper().str.replace("  ",' @ ')
# clean add home and away column 
df_spreads_2020[['Away Team', 'Home Team']] = df_spreads_2020['Matchup'].str.split(' @ ', expand=True)
# Remove leading/trailing whitespaces
df_spreads_2020['Away Team'] = df_spreads_2020['Away Team'].str.strip()
df_spreads_2020['Home Team'] = df_spreads_2020['Home Team'].str.strip()
#split to create away and home spread cols 
df_spreads_2020[['Away Spread', 'Home Spread']] = df_spreads_2020['Sides'].str.split(expand=True)
#split to create away and home consensus cols 
df_spreads_2020[['Away Consensus', 'Home Consensus']] = df_spreads_2020['Consensus'].str.split(expand=True)
# convert to dtype = float for consensus cols 
df_spreads_2020['Away Consensus'] =  df_spreads_2020['Away Consensus'].str.rstrip('%').astype(int)/100
df_spreads_2020['Home Consensus'] = df_spreads_2020['Home Consensus'].str.rstrip('%').astype(int)/100
#create consensus favorite vs. underdog cols 
df_spreads_2020['Con. Spread Favorite'] = df_spreads_2020.apply(lambda x: x['Away Team'] if x['Away Consensus'] > x['Home Consensus'] else x['Home Team'], axis=1)
df_spreads_2020['Con. Spread Underdog'] = df_spreads_2020.apply(lambda x: x['Away Team'] if x['Away Consensus'] < x['Home Consensus'] else x['Home Team'], axis=1)

# Define the dictionary mapping for replacements
replacement_dict = {'LV': 'Raiders',
                    'KC': 'Chiefs',
                    'NE': 'Patriots',
                    'GB': 'Packers',
                    'NO': 'Saints',
                    'BAL': 'Ravens',
                    'SF': '49ers',
                    'NYG': 'Giants',
                    'JAC': 'Jaguars',
                    'PIT': 'Steelers',
                    'TB': 'Buccaneers',
                    'CLE': 'Browns',
                    'IND': 'Colts',
                    'PHI': 'Eagles',
                    'ARI': 'Cardinals',
                    'WAS': 'Commanders',
                    'ATL': 'Falcons',
                    'NYJ': 'Jets',
                    'SEA': 'Seahawks',
                    'MIA': 'Dolphins',
                    'HOU': 'Texans',
                    'CAR': 'Panthers',
                    'CHI': 'Bears',
                    'CIN': 'Bengals',
                    'LAR': 'Rams',
                    'BUF': 'Bills',
                    'DET': 'Lions',
                    'DEN': 'Broncos',
                    'TEN': 'Titans',
                    'MIN': 'Vikings',
                    'LAC': 'Chargers',
                    'DAL': 'Cowboys'}
# Replace the values in the DataFrame column using map()
df_spreads_2020['Away Team'] = df_spreads_2020['Away Team'].map(replacement_dict)
df_spreads_2020['Home Team'] = df_spreads_2020['Home Team'].map(replacement_dict)
df_spreads_2020['Con. Spread Favorite'] = df_spreads_2020['Con. Spread Favorite'].map(replacement_dict)
df_spreads_2020['Con. Spread Underdog'] = df_spreads_2020['Con. Spread Underdog'].map(replacement_dict)
df_spreads_2020['Matchup'] = df_spreads_2020['Away Team'] + ' @ ' + df_spreads_2020['Home Team']
df_spreads_2020.tail()

,Matchup,Date,Consensus,Sides,Picks,Indepth,Away Team,Home Team,Away Spread,Home Spread,Away Consensus,Home Consensus,Con. Spread Favorite,Con. Spread Underdog
253,Chargers @ Chiefs,2021-01-03,48% 52%,-6.5 +6.5,562 616,Details,Chargers,Chiefs,-6.5,+6.5,0.48,0.52,Chiefs,Chargers
254,Ravens @ Bengals,2021-01-03,52% 48%,-13.5 +13.5,864 796,Details,Ravens,Bengals,-13.5,+13.5,0.52,0.48,Ravens,Bengals
255,Jets @ Patriots,2021-01-03,52% 48%,+3 -3,831 778,Details,Jets,Patriots,+3,-3,0.52,0.48,Jets,Patriots
256,Packers @ Bears,2021-01-03,51% 49%,-4.5 +4.5,893 846,Details,Packers,Bears,-4.5,+4.5,0.51,0.49,Packers,Bears
257,Seahawks @ 49ers,2021-01-03,51% 49%,-7 +7,850 806,Details,Seahawks,49ers,-7,+7,0.51,0.49,Seahawks,49ers


In [6]:
#import consensus data for spreads 
game_dates = list(df_scores_2020['Date'].unique())
uri_list = []
uri = "https://contests.covers.com/consensus/topoverunderconsensus/nfl/overall/"

for date in game_dates:
    uri_list.append(uri + date)

totals = []
for item in uri_list:
    try:
        res = requests.get(item)
        soup = BeautifulSoup(res.content, 'lxml')
        table = soup.find_all('table')[0]
        temp_df = pd.read_html(str(table))[0]  # Obtain the first table from the list
        temp_df['Date'] = re.search(r'\d{4}-\d{2}-\d{2}', item).group()  # Add the corresponding date column

        totals.append(temp_df)
    except IndexError:
        print(f"IndexError occurred for URL: {item}")
        continue
    except Exception as e:
        print(f"An error occurred for URL: {item}")
        print(f"Error details: {str(e)}")
        continue

df_totals_2020 = pd.concat(totals, ignore_index=True)
df_totals_2020.tail()

IndexError occurred for URL: https://contests.covers.com/consensus/topoverunderconsensus/nfl/overall/2020-12-02


,Matchup,Date,Consensus,Total,Picks,Indepth
251,NFL Jac Ind,2021-01-03,54 % Over 46 % Under,48.0,570 481,Details
252,NFL Sea Sf,2021-01-03,54 % Under 46 % Over,45.0,601 517,Details
253,NFL Gb Chi,2021-01-03,52 % Over 48 % Under,49.0,620 574,Details
254,NFL Pit Cle,2021-01-03,51 % Under 49 % Over,44.5,569 544,Details
255,NFL Was Phi,2021-01-03,51 % Under 49 % Over,43.5,529 508,Details


In [7]:
#clean matchup column 
df_totals_2020['Matchup'] = df_totals_2020['Matchup'].str[4:].str.upper().str.replace("  ",' @ ')
# clean add home and away column 
df_totals_2020[['Away Team', 'Home Team']] = df_totals_2020['Matchup'].str.split(' @ ', expand=True)
# Remove leading/trailing whitespaces
df_totals_2020['Away Team'] = df_totals_2020['Away Team'].str.strip()
df_totals_2020['Home Team'] = df_totals_2020['Home Team'].str.strip()
# create consensus columns 
df_totals_2020['Over Consensus'] = df_totals_2020['Consensus'].str.extract(r'(\d+) % Over').astype(int)/100
df_totals_2020['Under Consensus'] = df_totals_2020['Consensus'].str.extract(r'(\d+) % Under').astype(int)/100
#create consensus favorites vs underdogs 
df_totals_2020['Con. Total Favorite'] = df_totals_2020[['Over Consensus', 'Under Consensus']].apply(lambda x: 'Over' 
                                                                                                    if x['Over Consensus'] > x['Under Consensus'] else 'Under', axis=1)
df_totals_2020['Con. Total Underdog'] = df_totals_2020[['Over Consensus', 'Under Consensus']].apply(lambda x: 'Over' 
                                                                                                    if x['Over Consensus'] < x['Under Consensus'] else 'Under', axis=1)
# Replace the values in the DataFrame column using map() from our previously defined replacement_dict
df_totals_2020['Away Team'] = df_totals_2020['Away Team'].map(replacement_dict)
df_totals_2020['Home Team'] = df_totals_2020['Home Team'].map(replacement_dict)
df_totals_2020['Matchup'] = df_totals_2020['Away Team'] + ' @ ' + df_totals_2020['Home Team']
df_totals_2020.tail()

,Matchup,Date,Consensus,Total,Picks,Indepth,Away Team,Home Team,Over Consensus,Under Consensus,Con. Total Favorite,Con. Total Underdog
251,Jaguars @ Colts,2021-01-03,54 % Over 46 % Under,48.0,570 481,Details,Jaguars,Colts,0.54,0.46,Over,Under
252,Seahawks @ 49ers,2021-01-03,54 % Under 46 % Over,45.0,601 517,Details,Seahawks,49ers,0.46,0.54,Under,Over
253,Packers @ Bears,2021-01-03,52 % Over 48 % Under,49.0,620 574,Details,Packers,Bears,0.52,0.48,Over,Under
254,Steelers @ Browns,2021-01-03,51 % Under 49 % Over,44.5,569 544,Details,Steelers,Browns,0.49,0.51,Under,Over
255,Commanders @ Eagles,2021-01-03,51 % Under 49 % Over,43.5,529 508,Details,Commanders,Eagles,0.49,0.51,Under,Over


In [8]:
print(df_scores_2020.columns)
print(df_spreads_2020.columns)
print(df_totals_2020.columns)

Index(['Week', 'Date', 'Away Team', 'Away Score', 'Home Team', 'Home Score',
       'Final Score', 'Total Points', 'Winner', 'Away Score Differential',
       'Home Score Differential', 'Matchup', 'Home_Away_Winner'],
      dtype='object')
Index(['Matchup', 'Date', 'Consensus', 'Sides', 'Picks', 'Indepth',
       'Away Team', 'Home Team', 'Away Spread', 'Home Spread',
       'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
       'Con. Spread Underdog'],
      dtype='object')
Index(['Matchup', 'Date', 'Consensus', 'Total', 'Picks', 'Indepth',
       'Away Team', 'Home Team', 'Over Consensus', 'Under Consensus',
       'Con. Total Favorite', 'Con. Total Underdog'],
      dtype='object')


In [9]:
# pd.merge(df1, df2, on=['Key_Column_1', 'Key_Column_2'], how='inner')
combined_df = pd.merge(df_scores_2020[['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 
                                       'Winner', 'Home_Away_Winner', 'Away Team', 'Away Score', 
                                       'Away Score Differential', 'Home Team', 'Home Score', 'Home Score Differential']], 
                       df_spreads_2020[['Matchup', 'Date', 'Away Team','Away Spread', 'Home Team', 
                                        'Home Spread', 'Away Consensus', 'Home Consensus', 
                                        'Con. Spread Favorite','Con. Spread Underdog']],
                       on=['Matchup', 'Date', 'Away Team', 'Home Team'], 
                       how='inner')

In [10]:
final_df_nfl_2020 = pd.merge(combined_df[['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 'Winner',
                                           'Home_Away_Winner', 'Away Team', 'Away Score',
                                           'Away Score Differential', 'Home Team', 'Home Score',
                                           'Home Score Differential', 'Away Spread', 'Home Spread',
                                           'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
                                           'Con. Spread Underdog']],
                            df_totals_2020[['Matchup', 'Date', 'Away Team', 'Home Team', 'Total',
                                            'Over Consensus', 'Under Consensus',
                                            'Con. Total Favorite', 'Con. Total Underdog']],
                             on=['Matchup', 'Date', 'Away Team', 'Home Team'], 
                             how='inner')

final_df_nfl_2020['Win Marginal'] = np.abs(final_df_nfl_2020['Away Score'] - final_df_nfl_2020['Home Score'])

In [11]:
# 1) did away vs home team cover? -- Done 
# Convert 'Away Spread' and 'Home Spread' columns to numeric values
final_df_nfl_2020['Away Spread'] = pd.to_numeric(final_df_nfl_2020['Away Spread'])
final_df_nfl_2020['Home Spread'] = pd.to_numeric(final_df_nfl_2020['Home Spread'])

final_df_nfl_2020['Spread Covering Team'] = final_df_nfl_2020.apply(lambda row: row['Home Team'] 
                                                              if row['Home Score'] + row['Home Spread'] >= row['Away Score'] 
                                                              else row['Away Team'], axis=1)
final_df_nfl_2020[['Matchup', 'Final Score','Home Team', 'Home Spread', 'Home Score', 'Home Spread','Away Team', 'Away Spread', 'Spread Covering Team']].head()

,Matchup,Final Score,Home Team,Home Spread,Home Score,Home Spread,Away Team,Away Spread,Spread Covering Team
0,Texans @ Chiefs,20 - 34,Chiefs,-18.5,34,-18.5,Texans,18.5,Texans
1,Seahawks @ Falcons,38 - 25,Falcons,13.5,25,13.5,Seahawks,-13.5,Falcons
2,Jets @ Bills,17 - 27,Bills,-13.0,27,-13.0,Jets,13.0,Jets
3,Bears @ Lions,27 - 23,Lions,-10.5,23,-10.5,Bears,10.5,Bears
4,Packers @ Vikings,43 - 34,Vikings,9.5,34,9.5,Packers,-9.5,Vikings


In [12]:
# 2) Did over/under hit? -- Done

# Create "Totals Covered" column
final_df_nfl_2020['Totals Covering Line'] = final_df_nfl_2020.apply(lambda row: 'Over' 
                                          if row['Total Points'] > row['Total'] 
                                          else ('Under' if row['Total Points'] < row['Total'] else 'Push'), axis=1)

# Print the updated DataFrame
final_df_nfl_2020[['Matchup', 'Total Points', 'Total', 'Totals Covering Line']].head()

,Matchup,Total Points,Total,Totals Covering Line
0,Texans @ Chiefs,54,53.5,Over
1,Seahawks @ Falcons,63,49.0,Over
2,Jets @ Bills,44,39.5,Over
3,Bears @ Lions,50,42.5,Over
4,Packers @ Vikings,77,44.5,Over


In [13]:
# 3) Did consensus fav vs underdog win? [Spread] -- Done

# Create "Consensus Spread Winner" column
final_df_nfl_2020['Consensus Spread Winner'] = final_df_nfl_2020.apply(lambda row: 'Majority' 
                                            if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] > 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] > 0.5))
                                            else 'Minority' if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] < 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] < 0.5))
                                            else 'Push' if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] == 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] == 0.5))
                                            else '', axis=1)

# Print the updated DataFrame
final_df_nfl_2020[['Away Team', 'Home Team', 'Away Consensus', 'Home Consensus', 'Consensus Spread Winner']].head()

,Away Team,Home Team,Away Consensus,Home Consensus,Consensus Spread Winner
0,Texans,Chiefs,0.54,0.46,Majority
1,Seahawks,Falcons,0.57,0.43,Minority
2,Jets,Bills,0.30,0.70,Minority
3,Bears,Lions,0.50,0.50,Push
4,Packers,Vikings,0.48,0.52,Majority


In [14]:
# 4) Did consensus fav vs underdog win? [Totals] -- Done
# Create "Total Consensus Winner" column
final_df_nfl_2020['Total Consensus Winner'] = final_df_nfl_2020.apply(lambda row: 'Majority' 
                                                  if ((row['Totals Covering Line'] == 'Under' and row['Under Consensus'] > 0.5) or
                                                                   (row['Totals Covering Line'] == 'Over' and row['Over Consensus'] > 0.5))
                                                  else 'Minority' if ((row['Totals Covering Line'] == 'Under' and row['Under Consensus'] < 0.5) or
                                                                   (row['Totals Covering Line'] == 'Over' and row['Over Consensus'] < 0.5))
                                                  else 'Push' if (row['Under Consensus'] == 0.5) or (row['Over Consensus'] == 0.5)
                                                  else '', axis=1)

# Print the updated DataFrame
#final_df_nfl_2020[['Matchup', 'Total Points', 'Total', 'Totals Covering Line', 'Over Consensus', 'Away Consensus', 'Total Consensus Winner']]

In [15]:
final_df_nfl_2020.columns

Index(['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 'Winner',
       'Home_Away_Winner', 'Away Team', 'Away Score',
       'Away Score Differential', 'Home Team', 'Home Score',
       'Home Score Differential', 'Away Spread', 'Home Spread',
       'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
       'Con. Spread Underdog', 'Total', 'Over Consensus', 'Under Consensus',
       'Con. Total Favorite', 'Con. Total Underdog', 'Win Marginal',
       'Spread Covering Team', 'Totals Covering Line',
       'Consensus Spread Winner', 'Total Consensus Winner'],
      dtype='object')

In [16]:
final_df_nfl_2020

,Week,Date,Matchup,Final Score,Total Points,Winner,Home_Away_Winner,Away Team,Away Score,Away Score Differential,...,Total,Over Consensus,Under Consensus,Con. Total Favorite,Con. Total Underdog,Win Marginal,Spread Covering Team,Totals Covering Line,Consensus Spread Winner,Total Consensus Winner
0,1,2020-09-10,Texans @ Chiefs,20 - 34,54,Chiefs,Home,Texans,20,14,...,53.5,0.57,0.43,Over,Under,14,Texans,Over,Majority,Majority
1,1,2020-09-13,Seahawks @ Falcons,38 - 25,63,Seahawks,Away,Seahawks,38,-13,...,49.0,0.53,0.47,Over,Under,13,Falcons,Over,Minority,Majority
2,1,2020-09-13,Jets @ Bills,17 - 27,44,Bills,Home,Jets,17,10,...,39.5,0.43,0.57,Under,Over,10,Jets,Over,Minority,Minority
3,1,2020-09-13,Bears @ Lions,27 - 23,50,Bears,Away,Bears,27,-4,...,42.5,0.41,0.59,Under,Over,4,Bears,Over,Push,Minority
4,1,2020-09-13,Packers @ Vikings,43 - 34,77,Packers,Away,Packers,43,-9,...,44.5,0.44,0.56,Under,Over,9,Vikings,Over,Majority,Minority
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,17,2021-01-03,Chargers @ Chiefs,38 - 21,59,Chargers,Away,Chargers,38,-17,...,43.0,0.63,0.37,Over,Under,17,Chargers,Over,Minority,Majority
233,17,2021-01-03,Cardinals @ Rams,7 - 18,25,Rams,Home,Cardinals,7,11,...,41.5,0.55,0.45,Over,Under,11,Rams,Under,Majority,Minority
234,17,2021-01-03,Seahawks @ 49ers,26 - 23,49,Seahawks,Away,Seahawks,26,-3,...,45.0,0.46,0.54,Under,Over,3,49ers,Over,Minority,Minority
235,17,2021-01-03,Saints @ Panthers,33 - 7,40,Saints,Away,Saints,33,-26,...,46.5,0.56,0.44,Over,Under,26,Saints,Under,Majority,Minority


In [17]:
final_df_nfl_2020.to_csv('final_df_nfl_2020.csv')